In [ ]:
qb = QuantBook()
import pandas as pd
import numpy as np

from scipy import optimize
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

In [ ]:
# Optimization Framework

def efficient_frointier_on_sharp(cov_mat,exp_vec,rf):
    """
    Output: weights; sharp ratio
    """
    f = lambda x: -1*(exp_vec@x - rf)/np.sqrt(x@cov_mat@x.T)
    n = len(exp_vec)
    # boundary = ((0, 1) for _ in range(n))
    constraints = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}
    opt = optimize.minimize(f,np.ones(n)/n,constraints=constraints)
    return opt.x,-opt.fun

## risk parity portfolio construction
def port_vol_calculate(weights,covariance):
    variance = weights@covariance@weights.T
    return np.sqrt(variance)

def component_std_calculate(weights,covariance):
    port_vol = port_vol_calculate(weights,covariance)
    return weights*(covariance@weights.T)/port_vol

def component_std_sse_calculate(weights,covariance,budget=None):
    if not budget:
        budget = np.ones_like(weights)
    csd = component_std_calculate(weights,covariance)/budget
    scale_csd = csd - csd.mean()
    sse = scale_csd @ scale_csd.T

    return sse

def risk_parity_portfolio_on_sse(covariance,budget = None):
    n = covariance.shape[0]
    cons = {'type':'eq',"fun":lambda w:np.sum(w)-1}
    bounds = ((0,1) for i in range(n))
    opt_result = optimize.minimize(lambda w: 1e3*component_std_sse_calculate(w,covariance,budget),x0 = np.array([1/n]*n),constraints=cons,bounds=bounds)

    return opt_result.x

## construct the risk portfolio based on the es
def component_es(weights,returns,delta = 1e-6):
    n = len(weights)
    port_es = Return_ES(returns@weights.T)
    es_list = np.zeros(n)
    for i in range(n):
        ind_w = weights[i]
        weights[i] += delta
        es_list[i] = ind_w * (Return_ES(returns@weights.T)-port_es)/delta
        weights[i] = ind_w

    return es_list

def component_es_sse(weights,returns,budget,delta = 1e-6):
    """
    Budge should be within the list form.
    """
    if not budget:
        budget = np.ones_like(weights)
    ces= component_es(weights,returns,delta)/budget
    scale_com_es = ces - ces.mean()
    return scale_com_es @ scale_com_es.T

def risk_parity_port_es(returns,budget=None):
    n = returns.shape[1]
    cons = {'type':'eq','fun':lambda w:np.sum(w) - 1}
    bounds = ((0,1) for i in range(n))
    opt_result = optimize.minimize(lambda w: 1e5*component_es_sse(w,returns,budget),x0 = np.ones(n)/n,constraints=cons,bounds=bounds)

    return opt_result.x

def Return_ES(sim_x,alpha=0.05):
    order_x = np.sort(sim_x)
    n = alpha*len(order_x)
    up_n = int(np.ceil(n))
    dn_n = int(np.floor(n))
    VaR = (order_x[up_n+1]+order_x[dn_n-1])/2
    ES = -1*np.mean(order_x[order_x <= VaR])
    return ES


def calc_max_drawdown(pnl):
    pnl_array = pnl.to_numpy().flatten()
    drawdowns = []
    max_drawdown_duration = 0
    start = 0
    max_so_far = pnl_array[0]
    max_drawdown = 0
    length = len(pnl_array)
    for i in range(length):
        if pnl_array[i] > max_so_far:
            start = i
            drawdown = 0
            drawdowns.append(drawdown)
            max_so_far = pnl_array[i]
        else:
            drawdown = pnl_array[i] - max_so_far
            if drawdown <= max_drawdown:
                max_drawdown = drawdown
                max_drawdown_duration = max(max_drawdown_duration, i - start)
            drawdowns.append(drawdown/max_so_far)
    drawdown_array = pd.Series(drawdowns, index=pnl.index)
    return drawdown_array.min(), max_drawdown_duration

In [ ]:
# Risk Attribution Analysis Framework

def weights_update(weights,returns):
    # check dimensions
    if len(weights) != returns.shape[1]:
        raise ValueError('Dimensions of Returns and Weights do not match')
    
    new_weights = np.zeros((returns.shape[0],len(weights)))
    
    for i in range(returns.shape[0]):
        new_weights[i,:] = weights
        weights *= (1+returns[i,:])
        Rt = sum(weights)-1
        weights = weights/(Rt+1)

    return new_weights

def carinok_calculate(port_returns):
    totalreturn = (1+port_returns).prod(axis=0)-1
    k = np.log(1+totalreturn)/totalreturn
    
    de_no = port_returns.copy()
    de_no[de_no == 0] = 1
    cari_fact = np.log(1+port_returns)/(de_no*k)

    return cari_fact

"""
Total Return excluding portfolio
"""
def port_return_calculate(updated_factor_rets,resid_rets):
    total_rets = np.hstack((updated_factor_rets.values, resid_rets[:, np.newaxis]))
    total = np.exp(np.sum(np.log(total_rets+1),axis=0))-1
    return total

"""
Return attribution
"""

def return_attribution_calculate(weighted_returns):
    port_returns = weighted_returns.sum(axis=1)
    carinok = carinok_calculate(port_returns)
    return_attribution = carinok @ weighted_returns
    return_attribution = np.append(return_attribution,return_attribution.sum())
    return return_attribution

"""
Vol Attribution
"""
def risk_attribution_calculate(weighted_returns):
    port_returns = weighted_returns.sum(axis=1)
    risk = port_returns.std(ddof=1)

    n = weighted_returns.shape[1]
    risk_attribution = np.zeros(n+1)

    for i in range(n):
        model = sm.OLS(weighted_returns[:,i],sm.add_constant(port_returns))
        results = model.fit()

        risk_attribution[i] = results.params[1] * risk
    risk_attribution[n] = risk
    return risk_attribution

In [ ]:
# Pairs - To extract portfolio NAV from backtest results 
backtest = api.ReadBacktest(16513464, '12da9b60e487a26031f3ad3af3a6887e') 
chartpoint_ls = backtest.Charts["Strategy Equity"].Series["Equity"].Values 
nav_pair17 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_pair17 = pd.Series(nav_pair17,index=date)
pair17 = nav_pair17.pct_change().dropna()

backtest = api.ReadBacktest(16513464, '855fffa38804a69659f9fa0e98906b40') 
chartpoint_ls = backtest.Charts["Strategy Equity"].Series["Equity"].Values 
nav_pair22 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_pair22 = pd.Series(nav_pair22,index=date)
pair22 = nav_pair22.pct_change().dropna()

backtest = api.ReadBacktest(16513464, '5d793b15bf5d6e30c16712829470fa2d') 
chartpoint_ls = backtest.Charts["Strategy Equity"].Series["Equity"].Values 
nav_pair16 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_pair16 = pd.Series(nav_pair16,index=date)
pair16 = nav_pair16.pct_change().dropna()

In [ ]:
# a2c
backtest_e1 = api.ReadBacktest(16527795, '1956e15c56f7188cde0a6faf081cdd5b') 
chartpoint_ls = backtest_e1.Charts["Strategy Equity"].Series["Equity"].Values 
nav_a2c17 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_a2c17 = pd.Series(nav_a2c17,index=date)
a2c17 = nav_a2c17.pct_change().dropna()

backtest_e2 = api.ReadBacktest(16527795, '3b99b432a90e0def3b9f0593a2110b70')
chartpoint_ls = backtest_e2.Charts["Strategy Equity"].Series["Equity"].Values 
nav_a2c16 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_a2c16 = pd.Series(nav_a2c16,index=date)
a2c16 = nav_a2c16.pct_change().dropna()

backtest_e3 = api.ReadBacktest(16527795, 'e665a5ac4e5b4a241dff4cfdd82cd4c5')
chartpoint_ls = backtest_e3.Charts["Strategy Equity"].Series["Equity"].Values 
nav_a2c22 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_a2c22 = pd.Series(nav_a2c22,index=date)
a2c22 = nav_a2c22.pct_change().dropna()


In [ ]:
# CTA
backtest_q1 = api.ReadBacktest(16390435, '581ea4887e829b062d6c2794f80339d5') 
chartpoint_ls = backtest_q1.Charts["Strategy Equity"].Series["Equity"].Values 
nav_trend22 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_trend22 = pd.Series(nav_trend22,index=date)/10
trend22 = nav_trend22.pct_change().dropna()

backtest_q2 = api.ReadBacktest(16390435, '556a6229a3df0279155a081262cb4e91')
chartpoint_ls = backtest_q2.Charts["Strategy Equity"].Series["Equity"].Values 
nav_trend17 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_trend17 = pd.Series(nav_trend17,index=date)/10
trend17 = nav_trend17.pct_change().dropna()

backtest_q3= api.ReadBacktest(16390435, '3f74422f4e57b37fd319bfac1cdc6f52')
chartpoint_ls = backtest_q3.Charts["Strategy Equity"].Series["Equity"].Values 
nav_trend16 = [x.Close for x in chartpoint_ls if x.Time.hour != 5] 
date = [x.Time for x in chartpoint_ls if x.Time.hour != 5] 
nav_trend16 = pd.Series(nav_trend16,index=date)/10
trend16 = nav_trend16.pct_change().dropna()

# Correlation Map

In [ ]:
trend17 = trend17.resample('D').last().dropna()
a2c17 = a2c17.resample('D').last().dropna()
pair17 = pair17.resample('D').last().dropna()
df_17_21 = pd.concat([trend17,a2c17,pair17],axis=1)
df_17_21.columns = ['cs','rl','arb']

sns.set_theme(style="white")

# Compute the correlation matrix
corr = df_17_21.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(3, 4))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True, fmt=".2f")
plt.show()

# Sharpe

# 17-21

In [ ]:
nav_trend17 = nav_trend17.resample('D').last().dropna()
nav_a2c17 = nav_a2c17.resample('D').last().dropna()
nav_pair17 = nav_pair17.resample('D').last().dropna()
nav17 = (nav_trend17 + nav_a2c17 + nav_pair17)/3
pd.concat([nav_trend17, nav_a2c17, nav_pair17, nav17],axis=1).plot()
sharpe_trend17 = (nav_trend17.pct_change().mean()/nav_trend17.pct_change().std()) * np.sqrt(252)
sharpe_a2c17 = (nav_a2c17.pct_change().mean()/nav_a2c17.pct_change().std()) * np.sqrt(252)
sharpe_pair17 = (nav_pair17.pct_change().mean()/nav_pair17.pct_change().std()) * np.sqrt(252)
sharpe_nav17 = (nav17.pct_change().mean()/nav17.pct_change().std()) * np.sqrt(252)

In [ ]:
sharpe_nav17

In [ ]:
calc_max_drawdown(nav17)

## 22

In [ ]:
nav_trend22 = nav_trend22.resample('D').last().dropna()
nav_a2c22 = nav_a2c22.resample('D').last().dropna()
nav_pair22 = nav_pair22.resample('D').last().dropna()
nav22 = (nav_trend22 + nav_a2c22 + nav_pair22)/3
pd.concat([nav_trend22, nav_a2c22, nav_pair22, nav22],axis=1).plot()
plt.show()
sharpe_trend22 = (nav_trend22.pct_change().mean()/nav_trend22.pct_change().std()) * np.sqrt(252)
sharpe_a2c22 = (nav_a2c22.pct_change().mean()/nav_a2c22.pct_change().std()) * np.sqrt(252)
sharpe_pair22 = (nav_pair22.pct_change().mean()/nav_pair22.pct_change().std()) * np.sqrt(252)
sharpe_nav22 = (nav22.pct_change().mean()/nav22.pct_change().std()) * np.sqrt(252)

In [ ]:
t_nav = 0.5*nav_a2c22+0.5*nav_pair22
(t_nav.pct_change().mean()/t_nav.pct_change().std()) * np.sqrt(252)

pd.concat([nav_a2c22, nav_pair22, t_nav],axis=1).plot()
plt.show()

In [ ]:
sharpe_nav22

In [ ]:
calc_max_drawdown(nav22)

## 16-17

In [ ]:
nav_trend16 = nav_trend16.resample('D').last().dropna()
nav_a2c16 = nav_a2c16.resample('D').last().dropna()
nav_pair16 = nav_pair16.resample('D').last().dropna()
nav16 = (nav_trend16 + nav_a2c16 + nav_pair16)/3
pd.concat([nav_trend16, nav_a2c16, nav_pair16, nav16],axis=1).plot()
plt.show()
sharpe_trend16 = (nav_trend16.pct_change().mean()/nav_trend16.pct_change().std()) * np.sqrt(252)
sharpe_a2c16 = (nav_a2c16.pct_change().mean()/nav_a2c16.pct_change().std()) * np.sqrt(252)
sharpe_pair16 = (nav_pair16.pct_change().mean()/nav_pair16.pct_change().std()) * np.sqrt(252)
sharpe_nav16 = (nav16.pct_change().mean()/nav16.pct_change().std()) * np.sqrt(252)

In [ ]:
sharpe_nav16

In [ ]:
calc_max_drawdown(nav16)

# In-Sample Risk Attribution Analysis

In [ ]:
rtns = pd.concat([trend17, a2c17, pair17],axis=1)
rtns.columns = ['trend','a2c','pair']

In [ ]:
init_weights = np.array([1/3,1/3,1/3])
updated_weights = weights_update(init_weights,rtns.values)
weighted_rets = updated_weights * rtns.values
returnattribution = return_attribution_calculate(weighted_rets)
returnattribution

In [ ]:
risk_atrribution = risk_attribution_calculate(weighted_rets)
risk_atrribution

# Risk Parity Optimization

### Pure Risk Parity Optimization

In [ ]:
cov_mat = rtns.cov()
weights = risk_parity_portfolio_on_sse(cov_mat.values)
print('Weights for Trend, a2c, pair', weights)

## Risk Parity on Expected Shortfall Optimization

In [ ]:
cov_mat = rtns.cov()
weights = risk_parity_port_es(rtns)
print('Weights for Trend, a2c, pair', weights)

## apply the pure return

## 22.1-22.11

In [ ]:
rtns22 = pd.concat([trend22, a2c22, pair22],axis=1)
rtns22.columns = ['trend','a2c','pair']

nav22 = 0.30580517*nav_trend22 + 0.2744029*nav_a2c22 + 0.41979193*nav_pair22
pd.concat([nav_trend22, nav_a2c22, nav_pair22, nav22],axis=1).plot()
plt.show()

sharpe_nav22 = (nav22.pct_change().mean()/nav22.pct_change().std()) * np.sqrt(252)

In [ ]:
sharpe_nav22

## 16-17

In [ ]:
rtns16 = pd.concat([trend16, a2c16, pair16],axis=1)
rtns16.columns = ['trend','a2c','pair']

nav16 =  0.30580517*nav_trend16 + 0.2744029*nav_a2c16 + 0.41979193*nav_pair16
pd.concat([nav_trend16, nav_a2c16, nav_pair16, nav16],axis=1).plot()
plt.show()

sharpe_nav16 = (nav16.pct_change().mean()/nav16.pct_change().std()) * np.sqrt(252)

In [ ]:
sharpe_nav16